# Factors.jl
A package for factors/potentials with parametric supports

In [1]:
using DataFrames
using Factors

## Dimensions

The core unit are dimensions, which consist of names (`Symbol`) and a countably-finite supports:
* `ListDimension`
* `StepDimension`
* `UnitDimension`
* `CartesianDimension`

The latter 3 correspond to the following `Base.Range` types:
* `StepRange`
* `UnitRange`
* `OneTo`

In [2]:
l = ListDimension(:L, ["bob", "waldo", "superman"])

L:  ("bob","waldo","superman") (3)

In [3]:
s = StepDimension(:S, 'a', 2, 'z')

S:  'a':2:'y' (13)

In [4]:
u = UnitDimension(:U, 16, 88)

U:  16:88 (73)

In [5]:
c = CartesianDimension(:C, 16)

C:  1:16

Julia can decide the best Dimension type

In [6]:
xs = map(s -> dimension(:X, s), [('a', 'α'), 10:3:40, 2:15, 1:4, 16])

5-element Array{Any,1}:
 X:  ('a','α') (2)
 X:  10:3:40 (11) 
 X:  2:15 (14)    
 X:  1:4          
 X:  1:16         

In [7]:
map(typeof, xs)

5-element Array{Any,1}:
 Factors.ListDimension{Char}       
 Factors.StepDimension{Int64,Int64}
 Factors.UnitDimension{Int64}      
 Factors.CartesianDimension{Int64} 
 Factors.CartesianDimension{Int64} 

### Indexing and iterating

In [8]:
x = dimension(:X, 'α':'ω')

X:  'α':1:'ω' (25)

In [9]:
for v in x
    print(v, " ")
end

α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ ς σ τ υ φ χ ψ ω 

In [10]:
x[2]

'β'

In [11]:
indexin('β', x)

2

In [12]:
(I, d) = update(x, ['α', 'ψ', 'ζ', 'δ'])
I

4-element Array{Int64,1}:
  1
 24
  6
  4

In [13]:
d

X:  ('α','ψ','ζ','δ') (4)

### Dimension Comparisons

Equality for dimensions is by state values, not by type

In [14]:
ListDimension(:X, [1, 2, 3]) == StepDimension(:X, 1:1:3)  == UnitDimension(:X, 1:3) == CartesianDimension(:X, 3)

true

Comparisons use the position of elements in a dimension

In [15]:
o = ListDimension(:X, [3, 16, -2])
o .< -2 # here, 3 & 16 are less than -2

3-element BitArray{1}:
  true
  true
 false

In [16]:
# 3, 16, and -2 are all ≥ 3
o .≥ 3

3-element BitArray{1}:
 true
 true
 true

## Factors

A factor maps from the support of each of its dimensions to a `Float64`.

When constructing a Factor fom an array, the array is read in column-major order: the first dimension will correspond to the first axis in the array (column), etc ...

In [17]:
ft = Factor([:X, :Y], [1 4; 2 5; 3 6])

6 instantiations:
	X:  1:3
	Y:  1:2

There are multiple convience constructors:

In [18]:
c = CartesianDimension(:C, 3)

s = StepDimension(:S, 10, 2, 18)

Factor([c, s], rand(3, 5))
Factor(c, [2, 0, 16])
Factor([1 4; 2 5; 3 6], :X => 3:5, :Y => ['a', 'b'])

# creates CartesianDimensions
Factor([:X, :Y], rand(20, 16)) 
Factor(:X, [31, 33, 58])

# zero potentials
Factor(c)
Factor([c, s])
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'))
Factor(:A=>10, :B=>3:20)

# zero dimensional factor
Factor(2016)

1 instantiation: 2016.0

### Patterns

`pattern` returns the sequence of a dimension in a factor

In [22]:
c = CartesianDimension(:C, 3)
s = StepDimension(:S, 'a', 2, 'h')
ft = Factor([c, s])

pattern(ft)

12×2 Array{Int64,2}:
 1  1
 2  1
 3  1
 1  2
 2  2
 3  2
 1  3
 2  3
 3  3
 1  4
 2  4
 3  4

Also done via accessing that dimension

In [23]:
pattern_states(ft)

12×2 Array{Any,2}:
 1  'a'
 2  'a'
 3  'a'
 1  'c'
 2  'c'
 3  'c'
 1  'e'
 2  'e'
 3  'e'
 1  'g'
 2  'g'
 3  'g'

A Factors scope returns the names of the dimension. As does `names(ϕ)`

In [24]:
scope(ft)

2-element Array{Symbol,1}:
 :C
 :S

### Accessing

Iterating over a factor is done by dimensions

An `Assignment` can also select from the factor:

### Mapping

### Broadcasting

Operations can be broadcast along dimensions:

DataFrame(broadcast(*, ft, :B, [100, 0.01]))

Dimensions can be reduced.
Convience functions are provded for the following (As well as for their excited cousins, e.g. sum!):
* `sum`  
* `prod`  
* `maximum`  
* `minimum`  

### Joining

Factors can be joined through `join` or by multiplying (adding, etc.) them: